In [ ]:
#-*- coding: utf-8 -*-

<img align="right" style="max-width: 200px; height: auto" src="hsg_logo.png">

###  Lab 03 - "Supervised Machine Learning"

Introduction to AI and ML, University of St. Gallen, Autumn Term 2019

In the last lab, you learned about Python programming elements such as conditions, loops as well as how to implement functions etc. In this third lab, we will build our first supervised machine learning "pipeline" using:


>- (1) the **Gaussian Naive-Bayes (GNB)** classifier, and; 
>- (2) the **k Nearest-Neighbours (kNN)** classifier 

you learned about in the lecture.

The **Naive-Bayes (NB)** classifier belongs to the family of simple "probabilistic classifiers" based on applying Bayes' theorem with a strong (naive) independence assumptions between the features. Naive Bayes has been studied extensively since the 1950s and remains an accessible (baseline) method for text categorization as well as other domains.


The **k Nearest-Neighbours (kNN)** is a simple, easy to understand, versatile, but powerful machine learning algorithm. Until recently (prior to the advent of deep learning approaches) it was used in a variety of applications such as finance, healthcare, political science, handwriting detection, image recognition and video recognition. In Credit ratings, financial institutes will predict the credit rating of customers. 

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

### Lab Objectives:

After today's lab you should be able to:

> 1. Understand how a Gaussian **Naive-Bayes (NB)** classifier can be trained and evaluated.
> 2. Understand how a **k Nearest-Neighbor (kNN)** classifier can be trained and evaluated.
> 3. Know how to Python's sklearn library to **train** and **evaluate** arbitrary classifiers.
> 4. Understand how to **evaluate** and **interpret** the classification results.

Before we start let's watch a motivational video:

In [ ]:
from IPython.display import YouTubeVideo
# OpenAI: "Solving Rubik's Cube with a Robot Hand"
# YouTubeVideo('x4O8pojMF0w', width=1024, height=576)

### Step 0: Setup of the Analysis Environment

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. In this lab will use the pandas, numpy, sklearn, matplotlib and seaborn library. Let's import the libraries by the execution of the statements below:

In [ ]:
# import the numpy, scipy and pandas data science library
import pandas as pd
import numpy as np
from scipy.stats import norm

# import sklearn data and data pre-processing libraries
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

# import sklearn naive.bayes and k-nearest neighbor classifier library
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# import sklearn classification evaluation library
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib data visualization library
import matplotlib.pyplot as plt
import seaborn as sns

Enable inline Jupyter notebook plotting:

In [ ]:
%matplotlib inline

Set random seed of all our experiments:

In [ ]:
random_seed = 42

## A. Gaussian "Naive-Bayes" (NB) Classification

### Step 1.0: Dataset Download and Data Assessment

The **Iris Dataset** is a classic and straightforward dataset often used as a "Hello World" example in multi-class classification. This data set consists of measurements taken from three different types of iris flowers (referred to as **Classes**),  namely the Iris Setosa, the Iris Versicolour, and, the Iris Virginica) and their respective measured petal and sepal length (referred to as **Features**).

<img align="center" style="max-width: 700px; height: auto" src="iris_dataset.png">

(Source: http://www.lac.inpe.br/~rafael.santos/Docs/R/CAP394/WholeStory-Iris.html)

In total, the dataset consists of **150 samples** (50 samples taken per class) as well as their corresponding **4 different measurements** taken for each sample. Please, find below the list of the individual measurements:

>- `Sepal length (cm)`
>- `Sepal width (cm)`
>- `Petal length (cm)`
>- `Petal width (cm)`

Further details of the dataset can be obtained from the following puplication: *Fisher, R.A. "The use of multiple measurements in taxonomic problems" Annual Eugenics, 7, Part II, 179-188 (1936); also in "Contributions to Mathematical Statistics" (John Wiley, NY, 1950)."*

Let's load the dataset and conduct a preliminary data assessment: 

In [ ]:
iris = datasets.load_iris()

Print and inspect the names of the four features contained in the dataset:

In [ ]:
iris.feature_names

Determine and print the feature dimensionality of the dataset:

In [ ]:
iris.data.shape

Determine and print the class label dimensionality of the dataset:

In [ ]:
iris.target.shape

Print and inspect the names of the three classes contained in the dataset:

In [ ]:
iris.target_names

Let's briefly envision how the feature information of the dataset is collected and presented in the data:

<img align="center" style="max-width: 900px; height: auto" src="feature_collection.png">

Let's inspect the top five feature rows of the Iris Dataset:

In [ ]:
pd.DataFrame(iris.data, columns=iris.feature_names).head(5)

Let's also inspect the top five class labels of the Iris Dataset:

In [ ]:
pd.DataFrame(iris.target, columns=["class"]).head(5)

Let's now conduct a more in depth data assessment. Therefore, we plot the feature distributions of the Iris dataset according to their respective class memberships as well as the features pairwise relationships.

Pls. note that we use Python's **Seaborn** library to create such a plot referred to as **Pairplot**. The Seaborn library is a powerful data visualization library based on the Matplotlib. It provides a great interface for drawing informative statstical graphics (https://seaborn.pydata.org). 

In [ ]:
# init the plot
plt.figure(figsize=(10, 10))

# load the dataset also available in seaborn
iris_plot = sns.load_dataset("iris")

# plot a pairplot of the distinct feature distributions
sns.pairplot(iris_plot, diag_kind='hist', hue='species');

It can be observed from the created Pairplot, that most of the feature measurements that correspond to flower class "setosa" exhibit a nice **linear seperability** from the feature measurements of the remaining flower classes. In addition, the flower classes "versicolor" and "virginica" exhibit a commingled and **non-linear seperability** across all the measured feature distributions of the Iris Dataset.

### Step 1.1. Dataset Pre-Processing

To understand and evaluate the performance of any trained supervised machine learning model, it is good practice, to divide the dataset into a **training set** (the fraction of data records solely used for training purposes) and a **evaluation set** (the fraction of data records solely used for evaluation purposes). Pls. note, the **evaluation set** will never be shown to the model as part of the training process.

<img align="center" style="max-width: 500px; height: auto" src="train_eval_dataset.png">

We set the fraction of evaluation records to **30%** of the original dataset:

In [ ]:
eval_fraction = 0.3

Randomly split the dataset into training set and evaluation set using sklearns `train_test_split` function:

In [ ]:
# 70% training and 30% evaluation
x_train, x_eval, y_train, y_eval = train_test_split(iris.data, iris.target, test_size=eval_fraction, random_state=random_seed, stratify=None)

Evaluate the training set dimensionalities:

In [ ]:
x_train.shape, y_train.shape

Evaluate the evaluation set dimensionalities:

In [ ]:
x_eval.shape, y_eval.shape

### Step 1.2. Gaussian Naive-Bayes (NB) Classification

One popular (and remarkably simple) algorithm is the **Naive Bayes Classifier**. Note that one natural way to adress a given classification task is via the probabilistic question: **What is the most likely class $\hat{c}$ given a set of observations $x$?** Formally, we wish to output a prediction for $c$ by calculating its posterior probabilities $P(c|x)$ given the expression:

$$\hat{c} = \arg \max_{c} P(c|x)$$

In probability theory, the posterior probability referrs to the probability of observing a particular class $c$ after taking into consideration all the available information $x$. In our case this would unfortunately require that we estimate $P(c | \mathbf{x})$ for every value of $\mathbf{x} = x_1, x_2, ..., x_n$. 

Imagine that each feature could take one of just 2 values. For example, the feature $x_1 = 1$ might signify that the word pumpkin appears in a given document and $x_1 = 0$ would signify that it does not. If we had 30 such binary features, that would mean that we need to be prepared to calculate the probability $P(c | \mathbf{x})$ of any of $2^{30}$ (over 1 billion) possible values of the input vector $\mathbf{x}$. Moreover, where is the learning? If we need to see every single possible example in order to predict the corresponding label then we're not really learning a pattern but just memorizing the dataset.

One solution to this challenge is the so-called **Bayes' theorem** (alternatively Bayes' law or Bayes' rule) that you learned about in the lecture. It provides an elegant way of calculating class posterior probabilities $P(c|x)$ without the need of seeing every single possible example. Let's briefly revisit the formula of the Bayes' theorem below:

<img align="center" style="max-width: 400px; height: auto" src="bayes_theorem.png">

In the formula of the **Bayes' theorem** above,

>- $P(c|x)$ denotes the **posterior** probability of class $c$ given a set of features $x$ denoted by $x_1, x_2, ..., x_n$.
>- $P(c)$ denotes the **prior** probability of observing class $c$.
>- $P(x|c)$ denotes the **likelihood** which is the probability of a feature $x$ given class $c$.
>- $P(x)$ denotes the **evidence** which is the general probability of observing feature $x$.

Now, using the **chain rule** of probability, we can express the likelihood term $P( \mathbf{x} | c)$ as:

$$P( \mathbf{x} | c) = P(x_1 | c) \cdot P(x_2 | x_1, c) \cdot ... \cdot P( x_n | x_1, ..., x_{n-1}, c)$$

By itself, this expression doesn't get us any further. We still need (in a worst case) estimate roughly $2^d$ parameters. The trick of the (naive) Bayes' theorem however is to assume that the distinct features $x_1, x_2, ..., x_n$ are conditionally independent from each other given a particular class $c$. 

Using this assumption we're in much better shape, as the likelihood term $p( \mathbf{x} | c)$ term simplifies to: 

$$P( \mathbf{x} | c) = P(x_1 | c) \cdot P(x_2 | c) \cdot ... \cdot P( x_n | c) = \prod^n_i P(x_i | c)$$

Estimating each term in $\prod^n_i p(x_i | c)$ amounts to estimating just one feature distribution. So our assumption of conditional independence has taken the complexity of our model (in terms of the number of parameters) from an exponential dependence on the number of features to a linear dependence. Hence, we call it the **"naive"** Bayes' theorem, since it makes the naive assumption about feature independence, so we don't have to care about dependencies among them.

#### Step 1.2.1. Calculation of the prior probabilities $P(c)$ of each class

Let's get an intuition of the Bayes' theorem by first calculating the prior probability $P(c)$ of each class iris flower contained in the dataset. Therefore, we first obtain the number of occurance of each class in the extracted training data:

In [ ]:
# determine counts of unique class labels
unique, counts = np.unique(y_train, return_counts=True)

# concatenate counts and class labels in a python dictionary
class_counts = dict(zip(unique, counts))

# print obtained dictionary
print(class_counts)  

Let's convert the obtained counts into probabilites. Therefore, we divide the class counts by the overall number of observations contained in the extracted training data:

In [ ]:
# divide counts by the number of observations available in the training data
prior_probabilities = counts / np.sum(counts)

# print obtained probabilites
print(prior_probabilities)

#### Step 1.2.2. Calculation of the evidence $P(x)$ of each feature

Let's now calculate the general probability of observing feature $𝑥$ which denotes the evidence $P(x)$ of a feature.

During the lecture you learned that evidence distribution can be approximated by a Gaussian (Normal) probability distribution $\mathcal{N}(\mu, \sigma)$. This simplification is justified by the application of the "law of large numbers" or "Central Limit Theorem" (you may want to have a look at further details of the theorem under: https://en.wikipedia.org/wiki/Central_limit_theorem). In general, the probability density of a Gaussian "Normal" distribution, as defined by the formula below. It is parametrized its mean $\mu$ and corresponding standard deviation $\sigma$:

<img align="center" style="max-width: 500px; height: auto" src="evidence_calculation.png">

Using the "law of large numbers" we will approximate the probability density $P(x) \approx \mathcal{N}(x | \mu, \sigma)$ of each of each feature by a Gaussian. But how can this be achieved? 

Let's start by inspecting the true probability density of the **sepal length** feature (the first feature) of the Iris Dataset. The following line of code determines a histogram of the true **sepal length** feature value distribution and plots it accordingly:

In [ ]:
# determine a histogram of the "sepal length" feature value distribution
hist_probabilities, hist_edges = np.histogram(x_train[:, 0], bins=10, range=(0,10), density=True)

# print the histogram feature value probabilites
print(hist_probabilities)

# print the histogram edges
print(hist_edges)

Let's also plot the probability density accordingly:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot histogram of "sepal length" observations
ax.hist(x_train[:, 0], bins=10, range=(0, 10), density=True, color='g')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)
ax.set_ylim([0.0, 0.5])

# add plot title
ax.set_title('Distribution of the "Sepal Length" feature', fontsize=10);

How can we approximate the true probability density of the **sepal length** feature using a Gaussian distribution? Well, all we need to do is to calculate it's mean $\mu$ and standard deviation $\sigma$. Let's start by calculating the mean $\mu$ of the **sepal length** feature:

In [ ]:
# calculate the mean of the sepal length observations
mean_sepal_length = np.mean(x_train[:, 0])

# print the obtained mean
print(mean_sepal_length)

Let's continue by calculating the standard devition $\sigma$ of the **sepal length** feature:

In [ ]:
# calculate the standard deviation of the sepal length observations
std_sepal_length = np.std(x_train[:, 0])

# print the obtained standard deviation
print(std_sepal_length)

We can now determine the approximate Gaussian (Normal) probability density distribution $\mathcal{N}(\mu, \sigma)$ of the **sepal length** feature using the $\mu$ and $\sigma$ obtained above. Thereby, we will utilize the `pdf.norm` function available in the `scipy.stats` package:

In [ ]:
# calculate the probability density function of the Gaussian distribution
hist_gauss_sepal_length = norm.pdf(np.arange(0, 10, 0.1), mean_sepal_length, std_sepal_length)

# print obtained probabilities
print(hist_gauss_sepal_length)

Let's now plot the approximate Gaussian (Normal) probability density distribution $P(x) \approx \mathcal{N}(\mu, \sigma)$:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), hist_gauss_sepal_length, color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal length" observations
ax.hist(x_train[:, 0], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of the "Sepal Length" feature', fontsize=10);

In [ ]:
# determine mean and std of the "sepal width" feature
mean_sepal_width = np.mean(x_train[:, 1])
std_sepal_width = np.std(x_train[:, 1])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_sepal_width, std_sepal_width), color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal width" observations
ax.hist(x_train[:, 1], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of the "Sepal Width" feature', fontsize=10);

In [ ]:
# determine mean and std of the "petal length" feature
mean_petal_length = np.mean(x_train[:, 2])
std_petal_length = np.std(x_train[:, 2])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_petal_length, std_petal_length), color='orange', linestyle='--', linewidth=2)

# plot histogram of "petal length" observations
ax.hist(x_train[:, 2], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of the "Petal Length" feature', fontsize=10);

In [ ]:
# determine mean and std of the "petal width" feature
mean_petal_width = np.mean(x_train[:, 3])
std_petal_width = np.std(x_train[:, 3])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_petal_width, std_petal_width), color='orange', linestyle='--', linewidth=2)

# plot histogram of "petal width" observations
ax.hist(x_train[:, 3], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of the "Petal Width" feature', fontsize=10);

#### Step 1.2.3. Calculation of the likelihood $P(x|c)$ of each feature

Let's now see how we can calculate the **likelihood** $P(x|c)$ which is the probability density of a feature given a certain class $c$. We will again can estimate $P(x|c)$ by a Gaussian (Normal) probability distribution $\mathcal{N}(\mu, \sigma)$ applying the "law of large numbers".

The **likelihood** probability density of a Gaussian "Normal" distribution, as defined by the formula below, is determined by its mean $\mu$, standard deviation $\sigma$ and it's corresponding class condition $c$:

<img align="center" style="max-width: 500px; height: auto" src="likelihood_calculation.png">

Let's start by applying the class conditioning. This is usually done by filtering the dataset for each class $c$:

In [ ]:
# collect all iris setosa measurements, class label = 0
x_train_setosa = x_train[y_train == 0]

# collect all iris versicolor measurements, class label = 1
x_train_versicolor = x_train[y_train == 1]

# collect all iris virginica measurements, class label = 2
x_train_virginica = x_train[y_train == 2]

Let's start by inspecting the true probability density of the **sepal length** feature (the first feature) of the iris dataset given the class **setosa**. The following line of code determines a histogram of the true feature value distribution:

In [ ]:
# determine a histogram of the "sepal length" feature value distribution given the class "setosa"
hist_setosa, bin_edges_setosa = np.histogram(x_train_setosa[:, 0], bins=10, range=(0, 10), density=True)

# print the histogram feature value probabilites
print(hist_setosa)

# print the histogram edges
print(bin_edges_setosa)

Let's also plot the probability density accordingly:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot histogram of "sepal length" observations given the class "setosa"
ax.hist(x_train_setosa[:, 0], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)
ax.set_ylim([0.0, 1.5])

# add plot title
ax.set_title('Distribution of the "Sepal Length" feature given class "Setosa"', fontsize=10);

We are again able to determine the approximate Gaussian (Normal) probability density distribution $\mathcal{N}(\mu, \sigma, c)$ of the **sepal length** feature given the class **setosa** using the $\mu$ and $\sigma$ obtained above as well as the `pdf.norm` function of the `scipy.stats` package.

Let's continue by calculating the mean $\mu$ of the **sepal length** feature given the class **setosa**:

In [ ]:
# calculate the mean of the sepal length observations given class "setosa"
mean_sepal_length_setosa = np.mean(x_train_setosa[:, 0])

# print the obtained mean
print(mean_sepal_length_setosa)

Let's continue by calculating the standard devition $\sigma$ of the **sepal length** feature given the class **setosa**:

In [ ]:
# calculate the standard deviation of the sepal length observations given class "setosa"
std_sepal_length_setosa = np.std(x_train_setosa[:, 0])

# print the obtained standard deviation
print(std_sepal_length_setosa)

In [ ]:
# calculate the probability density function of the Gaussian distribution
hist_gauss_sepal_length_setosa = norm.pdf(np.arange(0, 10, 0.1), mean_sepal_length_setosa, std_sepal_length_setosa)

# print obtained probabilities
print(hist_gauss_sepal_length_setosa)

Let's now plot the approximate Gaussian (Normal) probability density distribution $P(x | c) \approx \mathcal{N}(\mu, \sigma, c)$:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), hist_gauss_sepal_length_setosa, color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal length" observations given the class "setosa"
ax.hist(x_train_setosa[:, 0], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of "Sepal Length" feature given class "Setosa"', fontsize=10);

In [ ]:
# determine mean and std of the "sepal width" feature given class setosa
mean_sepal_width_setosa = np.mean(x_train_setosa[:, 1])
std_sepal_width_setosa = np.std(x_train_setosa[:, 1])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_sepal_width_setosa, std_sepal_width_setosa), color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal length" observations given the class "setosa"
ax.hist(x_train_setosa[:, 1], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of "Sepal Width" feature given class "Setosa"', fontsize=10);

In [ ]:
# determine mean and std of the "petal length" feature given class setosa
mean_petal_length_setosa = np.mean(x_train_setosa[:, 2])
std_petal_length_setosa = np.std(x_train_setosa[:, 2])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_petal_length_setosa, std_petal_length_setosa), color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal length" observations given the class "setosa"
ax.hist(x_train_setosa[:, 2], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of "Sepal Width" feature given class "Setosa"', fontsize=10);

In [ ]:
# determine mean and std of the "petal width" feature given class setosa
mean_petal_width_setosa = np.mean(x_train_setosa[:, 3])
std_petal_width_setosa = np.std(x_train_setosa[:, 3])

# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot fitted "gaussian" or normal distribution
ax.plot(np.arange(0, 10, 0.1), norm.pdf(np.arange(0, 10, 0.1), mean_petal_width_setosa, std_petal_width_setosa), color='orange', linestyle='--', linewidth=2)

# plot histogram of "sepal length" observations given the class "setosa"
ax.hist(x_train_setosa[:, 3], bins=10, range=(0, 10), density=True, color='green')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("$x$", fontsize=10)
ax.set_ylabel("$P(x)$", fontsize=10)

# add plot title
ax.set_title('Gaussian Approximation of "Sepal Width" feature given class "Setosa"', fontsize=10);

Now we have determined all the necessary probabilities and probability distributions $P(c)$, $P(x)$ and $P(x|c)$ to determine if a so far unseen **iris flower** observation corresponds to class **setosa**. Let's calculate the probability of two random observations of beeing of class **setosa**: 

In [ ]:
# init a random feature observation 
sepal_length = 5.8 
sepal_width  = 3.5
petal_length = 1.5
petal_width  = 0.25

# calculate the distinct elements of the Bayes theorem formula

# init the prior probability
prior = 0.3333 

# determine the likelihood probability
likelihood = norm.pdf(sepal_length, mean_sepal_length_setosa, std_sepal_length_setosa) * norm.pdf(sepal_width, mean_sepal_width_setosa, std_sepal_width_setosa) * norm.pdf(petal_length, mean_petal_length_setosa, std_petal_length_setosa) * norm.pdf(petal_width, mean_petal_width_setosa, std_petal_width_setosa)

# determine the evidence probability
evidence = norm.pdf(sepal_length, mean_sepal_length, std_sepal_length) * norm.pdf(sepal_width, mean_sepal_width, std_sepal_width) * norm.pdf(petal_length, mean_petal_length, std_petal_length) * norm.pdf(petal_width, mean_petal_width, std_petal_width)

# determine the posterior probability
posterior = (prior * likelihood) / evidence

# print the obtained posterior probability
print(posterior)

Ok, it seems to be likely that this Iris flower is of class **setosa**.

Let's compare this to a another **iris flower** observation and determine its probability of beeing of class **setosa**:

In [ ]:
# init a second random feature observation 
sepal_length = 7.8
sepal_width  = 2.3
petal_length = 6.4
petal_width  = 2.5

# calculate the distinct elements of the Bayes theorem formula

# init the prior probability
prior = 0.3333 

# determine the likelihood probability
likelihood = norm.pdf(sepal_length, mean_sepal_length_setosa, std_sepal_length_setosa) * norm.pdf(sepal_width, mean_sepal_width_setosa, std_sepal_width_setosa) * norm.pdf(petal_length, mean_petal_length_setosa, std_petal_length_setosa) * norm.pdf(petal_width, mean_petal_width_setosa, std_petal_width_setosa)

# determine the evidence probability
evidence = norm.pdf(sepal_length, mean_sepal_length, std_sepal_length) * norm.pdf(sepal_width, mean_sepal_width, std_sepal_width) * norm.pdf(petal_length, mean_petal_length, std_petal_length) * norm.pdf(petal_width, mean_petal_width, std_petal_width)

# determine the posterior probability
posterior = (prior * likelihood) / evidence

# print the obtained posterior probability
print(posterior)

Alright, this observations seems to be very improbable of class **setosa**.

#### Step 1.2.4. Training and utilization of a Gaussian Naive-Bayes Classifier using Python's Sklearn library

Luckily, there is a Python library named `sklearn` (https://scikit-learn.org) that provides a variety of machine learning algorithms that can be easily interfaced using the Python programming language. It also contains supervised classification algorithms such as the **Gaussian Naive-Bayes** classifier which we can use of the shelf.

Let's use `sklearn` and instantiate the **Gaussian Naive-Bayes** classifier:

In [ ]:
# init the Gaussian Naive Bayes classifier
gnb = GaussianNB(priors=None, var_smoothing=1e-09)

Train or fit the Gaussian Naive-Bayes classifier using the training dataset features and labels:

In [ ]:
# train the Gaussian Naive Bayes classifier
gnb.fit(x_train, y_train)

Utilize the trained model to predict the classes of the distinct observations contained in the evaluation dataset:

In [ ]:
y_pred = gnb.predict(x_eval)

Let's have a look at the class labels **predicted** by the Gaussian Naive-Bayes classifier on the evaluation dataset:

In [ ]:
y_pred

As well as the **true** class labels as contained in the evaluation dataset:

In [ ]:
y_eval

Determine the **prediction accuracy** of the trained model on the evaluation dataset:

In [ ]:
print("Accuracy: ", metrics.accuracy_score(y_eval, y_pred))

Determine number of **missclassified** data sampels in the evaluation dataset:

In [ ]:
print("Number of mislabeled points out of a total {} points: {}".format(x_eval.shape[0], np.sum(y_eval != y_pred)))

In the field of machine learning and in particular the field of statistical classification, a **confusion matrix**, also known as an error matrix, is a specific table layout that allows visualization of the performance of an algorithm. Each row of the matrix represents the number of instances that the classifier predicted per class, while each column represents the instances of the true or actual class:

<img align="center" style="max-width: 300px; height: auto" src="confusion_matrix.png">

(Source: https://en.wikipedia.org/wiki/Confusion_matrix)

Determine and plot the **confusion matrix** of the individual predictions:

In [ ]:
# determine the prediction confusion matrix
mat = confusion_matrix(y_eval, y_pred)

Plot the **confusion matrix** of the individual predictions:

In [ ]:
# init the plot
plt.figure(figsize=(5, 5))

# plot confusion matrix heatmap
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='BuGn_r', xticklabels=iris.target_names, yticklabels=iris.target_names)

# add plot axis labels
plt.xlabel('[true label]')
plt.ylabel('[predicted label]')

# add plot title
plt.title('Gaussian NB Confusion Matrix');

### Exercises:

We recommend you to try the following exercises as part of the lab:

**1. Train and evaluate the prediction accuracy of different train- vs. eval-data ratios.**

> Change the ratio of training data vs. evaluation data to 30%/70% (currently 70%/30%), fit your model and calculate the new classification accuracy. Subsequently, repeat the experiment a second time using a 10%/90% fraction of training data/evaluation data. What can be observed in both experiments in terms of classification accuracy? 

**2. Calculate the true-positive as well as the false-positive rate of the Iris versicolor vs. virginica.**

> Calculate the true-positive rate as well as false-positive rate of (1) the experiment exhibiting a 30%/70% ratio of training data vs. evaluation data and (2) the experiment exhibiting a 10%/90% ratio of training data vs. evaluation data.

## B. k Nearest-Neighbor (kNN) Classification

Now, let's have closer into a non-parametric method used for supervised classification tasks referred to as the the **k Nearest-Neighbors (kNN)** algorithm. As you learned during the in lecture k-NN classification, the output of the classifier is a class membership. 

Prior to running **k Nearest-Neighbor (kNN)** classification let's briefly revisit the distinct steps of the algorithm as discussed in the lecture:
<img align="center" style="max-width: 600px; height: auto" src="hsg_knn.png">

An object is classified by a majority vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

### Step 2.1: Dataset Download and Data Assessment

Let's try the k Nearest-Neighbour algorithm using the delcious **"Wine"** dataset is a classic and very easy multi-class classification dataset. 

The data is the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators (types). The dataset consists of in total **178 wines** as well as their corresponding **13 different measurements** taken for different constituents found in the three types of wine. Please, find below the list of the individual measurements (features):

>- `Alcohol`
>- `Malic acid`
>- `Ash`
>- `Alcalinity of ash`
>- `Magnesium`
>- `Total phenols`
>- `Flavanoids`
>- `Nonflavanoid phenols`
>- `Proanthocyanins`
>- `Color intensity`
>- `Hue`
>- `OD280/OD315 of diluted wines`
>- `CProline`

Further details on the dataset can be obtained from the following puplication: *Forina, M. et al, PARVUS - "An Extendible Package for Data Exploration, Classification and Correlation.", Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.*

Let's load the dataset and conduct a preliminary data assessment: 

In [ ]:
wine = datasets.load_wine()

Print and inspect feature names of the dataset:

In [ ]:
wine.feature_names

Print and inspect the class names of the dataset:

In [ ]:
wine.target_names

Print and inspect the top 10 feature rows of the dataset:

In [ ]:
pd.DataFrame(wine.data).head(10)

Print and inspect the top 10 labels of the dataset:

In [ ]:
pd.DataFrame(wine.target).head(10)

Determine and print the feature dimensionality of the dataset:

In [ ]:
wine.data.shape

Determine and print the label dimensionality of the dataset:

In [ ]:
wine.target.shape

Plot the data distributions of the distinct features:

In [ ]:
# init the plot
plt.figure(figsize=(10, 10))

# prepare the dataset to be plotable using seaborn

# convert to Panda's DataFrame
wine_plot = pd.DataFrame(wine.data, columns=wine.feature_names)

# add class labels to the DataFrame
wine_plot['class'] = wine.target

# plot a pairplot of the distinct feature distributions
sns.pairplot(wine_plot, diag_kind='hist', hue='class');

### 2.2. Dataset Pre-Processing

#### 2.2.1 Data Normalization

In [ ]:
# min-max normalize the distinct feature values
wine_data_norm = normalize(wine.data, norm='max', axis=0)

Print and inspect the top 10 feature rows of the normalized dataset:

In [ ]:
pd.DataFrame(wine_data_norm).head(10)

#### 2.2.2 Extraction of Training- and Evaluation-Dataset

To understand and evaluate the performance of any trained supervised machine learning model, it is good practice, to divide the dataset into a **training set** (the fraction of data records solely used for training purposes) and a **evaluation set** (the fraction of data records solely used for evaluation purposes). Pls. note, the **evaluation set** will never be shown to the model as part of the training process.

<img align="center" style="max-width: 500px; height: auto" src="train_eval_dataset.png">

We set the fraction of testing records to **30%** of the original dataset:

In [ ]:
eval_fraction = 0.3

Randomly split the dataset into training set and evaluation set using sklearns `train_test_split` function:

In [ ]:
# 70% training and 30% evaluation
X_train, X_eval, y_train, y_eval = train_test_split(wine_data_norm, wine.target, test_size=eval_fraction)

Evaluate the training set dimensionality:

In [ ]:
X_train.shape, y_train.shape

Evaluate the evaluation set dimensionality:

In [ ]:
X_eval.shape, y_eval.shape

### 2.3. k Nearest-Neighbor (kNN) Classification

#### 3.1 Nearest Neighbors Classification, k=5

Set the number of neighbors `k` to be considered in the classification of each sample: 

In [ ]:
k_nearest_neighbors = 5

Set the metric used in calculating the distances $D(x, x_i)$ between a sample $x$ and it's neighbors $x_i$. We will use the Euclidean distance that you learned about in the lecture, given by $\sqrt{(\sum^n_{i=1}((x - x_i)^2))}$:

In [ ]:
distance_metric = 'euclidean'

Init the **kNN classifier** of Python's `sklearn` libary of data science algoritms: 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_nearest_neighbors, metric=distance_metric)

Train the k-NN classifier using the training dataset:

In [ ]:
knn.fit(X_train, y_train);

Utilize the trained model to predict the response for the evaluation dataset:

In [ ]:
y_pred = knn.predict(X_eval)

Let's have a look at the predicted class labels:

In [ ]:
y_pred

As well as the true class labels:

In [ ]:
y_eval

Determine **prediction accuracy** of the trained model on the evaluation dataset:

In [ ]:
print("Accuracy, k=5: ", metrics.accuracy_score(y_eval, y_pred))

Determine and plot the **confusion matrix** of the individual predictions:

In [ ]:
# determine the prediction confusion matrix
mat = confusion_matrix(y_eval, y_pred)

Plot confusion matrix of the individual predictions:

In [ ]:
# init the plot
plt.figure(figsize=(5, 5))

# plot confusion matrix heatmap
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='BuGn_r', xticklabels=wine.target_names, yticklabels=wine.target_names)

# add plot axis labels
plt.xlabel('[true label]')
plt.ylabel('[predicted label]')

# add plot title
plt.title('k-NN Confusion Matrix, k=8');

Remember that as part of the lecture you learned about several measures to evaluate the quality of a retrieval system, namely **Precision**, **Recall** and **F1-Score**. Let's briefly revisit their definition and subsequently calculate those measures based on the confusion matrix above:

>- The **Precision**, denoted by Precision $=\frac{TP}{TP + FP}$, is the probability that a retrieved document is relevant.
>- The **Recall**, denoted by Recall $=\frac{TP}{TP + FN}$, is the probability that a relevant document is retrieved.
>- The **F1-Score**, denoted by F1-Score $= 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$, combines precision and recall is the harmonic mean of both measures.

In [ ]:
print(classification_report(y_eval, y_pred))

#### 3.2 Nearest Neighbors Classification, k=8

Set the number of neighbors `k` to be considered in the classification of each sample: 

In [ ]:
k_nearest_neighbors = 8

Init the **k-NN classifier** of Python's `sklearn` libary of data science algoritms: 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_nearest_neighbors)

Train the k-NN classifier using the training dataset:

In [ ]:
knn.fit(X_train, y_train);

Utilize the trained model to predict the response for the evaluation dataset:

In [ ]:
y_pred = knn.predict(X_eval)

Determine **prediction accuracy** of the trained model on the evaluation dataset:

In [ ]:
print("Accuracy, k=8: ", metrics.accuracy_score(y_eval, y_pred))

Determine and plot the **confusion matrix** of the individual predictions:

In [ ]:
# determine the prediction confusion matrix
mat = confusion_matrix(y_eval, y_pred)

Plot **confusion matrix** of the individual predictions:

In [ ]:
# init the plot
plt.figure(figsize=(5, 5))

# plot confusion matrix heatmap
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='BuGn_r', xticklabels=wine.target_names, yticklabels=wine.target_names)

# add plot axis labels
plt.xlabel('[true label]')
plt.ylabel('[predicted label]')

# add plot title
plt.title('k-NN Confusion Matrix, k=8');

Let's also obtain and compare the **Precision**, **Recall**, and, **F1-Score** of the `k=8` model:

In [ ]:
print(classification_report(y_eval, y_pred))

### Exercises:

We recommend you to try the following exercises as part of the lab:

**1. Train and evaluate the prediction accuracy of the k=1,...,40 Nearest Neighbor models.**

> Write a Python loop that trains and evaluates the prediction accuracy of all k-Nearest Neighbor parametrizations ranging from k=1,...,40. Collect and print the prediction accuracy of each model respectively and compare the results. What kind of behaviour in terms of prediction accuracy can be observed with increasing k?

**2. Plot the prediction accuracy of the k=1,...,40 Nearest Neighbor models.**

> Plot the prediction accuracy collected for each model above. The plot should display the distinct values of k at the x-axis and the corresponding model prediction accuracy on the y-axis.

**3. Train, evaluate and plot the prediction accuracy of the k=1,...,124 Nearest Neighbor models.**

> Train, evaluate and plot the prediction accuracy of all k-Nearest Neighbor parametrizations ranging from k=1,...,124. Collect and print the prediction accuracy of each model respectively and compare the results. What kind of behaviour in terms of prediction accuracy can be observed with increasing k?

### Lab Summary:

In this third lab, a step by step introduction into (1) **Gaussian Naive-Bayes (GNB)** and (2) **k Nearest-Neighbor (kNN)** classification is presented. The code and exercises presented in this lab may serves as a starting point for more complex and tailored programs.

You may want to execute the content of your lab outside of the Jupyter notebook environment, e.g. on a compute node or a server. The cell below converts the lab notebook into a standalone and executable python script. Pls. note that to convert the notebook, you need to install Python's **nbconvert** library and its extensions:

In [ ]:
# installing the nbconvert library
!pip install nbconvert
!pip install jupyter_contrib_nbextensions

Let's now convert the Jupyter notebook into a plain Python script:

In [ ]:
!jupyter nbconvert --to script aiml_lab_03.ipynb